# Model_Training

##### 1.1 Import Machining Libraries

In [1]:
# Basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Modeling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Importing models
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#### import data into csv

In [2]:
df = pd.read_csv('Data/studentsPerformance.csv')

## Preparing X and Y features

In [3]:
x = df.drop(columns=['math score'], axis=1)

In [4]:
x.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [5]:
print(x.shape)

(1000, 7)


In [6]:
y = df['math score']

In [7]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [8]:
# Creating column transformer with 3 transformers
num_features = x.select_dtypes(exclude=['object']).columns
cat_features = x.select_dtypes(include=['object']).columns

numeric_transformer = StandardScaler()
ohe_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', ohe_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_features)
    ]
)

In [9]:
X = preprocessor.fit_transform(x)

X.shape

(1000, 19)

In [10]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [11]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 19) (800,)
(200, 19) (200,)


#### Create Evaluate Function to give all metrics after training models

In [12]:
# create models to see which one is the best
models = {
    'K-NeighborsRegressor': KNeighborsRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor(verbose=0),
}


In [13]:
model_list = []
r2_train_list = []
r2_test_list = []


In [14]:
# Define the evaluate_model function
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    return mae, rmse, r2  # Corrected to return RMSE

In [15]:

# Loop through the models and evaluate them
for model_name, model in models.items():
    # Train model
    model.fit(X_train, y_train)

    # Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # --- Debugging Checks ---
    print(f"--- Evaluating {model_name} ---")
    print("Shape of y_train:", y_train.shape)
    print("Shape of y_train_pred:", y_train_pred.shape)
    print("Shape of y_test:", y_test.shape)
    print("Shape of y_test_pred:", y_test_pred.shape)

    print("Data type of y_train:", y_train.dtype)
    print("Data type of y_train_pred:", y_train_pred.dtype)
    print("Data type of y_test:", y_test.dtype)
    print("Data type of y_test_pred:", y_test_pred.dtype)
    # --- End of Debugging Checks ---

    # Evaluate the model
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Print the evaluation metrics
    print(model_name)
    model_list.append(model_name)

    print("Model Performance for Train Data")
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    r2_train_list.append(model_train_r2)

    print("---------------------------------------")

    print("Model Performance for Test Data")
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_test_list.append(model_test_r2)

    print("=" * 35)
    print("\n")

--- Evaluating K-NeighborsRegressor ---
Shape of y_train: (800,)
Shape of y_train_pred: (800,)
Shape of y_test: (200,)
Shape of y_test_pred: (200,)
Data type of y_train: int64
Data type of y_train_pred: float64
Data type of y_test: int64
Data type of y_test_pred: float64
K-NeighborsRegressor
Model Performance for Train Data
- Root Mean Squared Error: 5.7088
- Mean Absolute Error: 4.5177
- R2 Score: 0.8554
---------------------------------------
Model Performance for Test Data
- Root Mean Squared Error: 7.2494
- Mean Absolute Error: 5.6090
- R2 Score: 0.7840


--- Evaluating DecisionTreeRegressor ---
Shape of y_train: (800,)
Shape of y_train_pred: (800,)
Shape of y_test: (200,)
Shape of y_test_pred: (200,)
Data type of y_train: int64
Data type of y_train_pred: float64
Data type of y_test: int64
Data type of y_test_pred: float64
DecisionTreeRegressor
Model Performance for Train Data
- Root Mean Squared Error: 0.2795
- Mean Absolute Error: 0.0187
- R2 Score: 0.9997
-----------------------